# LanceDB Experiment Example

## Installations

In [ ]:
# !pip install --quiet --force-reinstall prompttools lancedb

## No setup required

In [1]:
from prompttools.experiment import LanceDBExperiment

## Run an experiment

One common use case is to compare two different embedding functions and how it may impact your document retrieval. We have can define what embedding functions we'd like to test here.

Note: If you previously haven't downloaded these embedding models. This may kick off downloads.

In [3]:
from sentence_transformers import SentenceTransformer
import openai
import os

# Configuring the environment variable OPENAI_API_KEY
if "OPENAI_API_KEY" not in os.environ:
    # OR set the key here as a variable
    openai.api_key = "..."


DEFAULT = SentenceTransformer("paraphrase-MiniLM-L3-v2")
MIMNILM_L6 = SentenceTransformer("all-MiniLM-L6-v2")


def default_embed_func(batch):
    return [DEFAULT.encode(sentence) for sentence in batch]


def minilm_l6_embed_func(batch):
    return [MIMNILM_L6.encode(sentence) for sentence in batch]


def openai_ada2_embed_func(batch):
    rs = openai.Embedding.create(input=batch, engine="text-embedding-ada-002")
    return [record["embedding"] for record in rs["data"]]


emb_fns = {"minilm_l6": minilm_l6_embed_func, "default": default_embed_func}
# Try with openai
# emb_fns = {"openai-ada-002": openai_ada2_embed_func, "minilm_l6": minilm_l6_embed_func,  "default": default_embed_func }

Next, we create our test inputs. In this case, we would like to create a new ChromaDB collection.

During the experiment, for each embedding function, a new ChromaDB collection will be temporarily created. The documents will be added into it. Then, we will query from it and examine the results.

In [4]:
import pandas as pd

use_existing_table = False  # Specify that we want to create a collection during the experiment

# Documents that will be added into the database. LanceDB also accepts other dataset formats like pydict, pyarrow, Pydantic etc.
# Learn more here - https://lancedb.github.io/lancedb/guides/tables/

data = pd.DataFrame(
    {
        "text": ["This is a document", "This is another document", "This is the document."],
        "metadatas": [{"source": "my_source"}, {"source": "my_source"}, {"source": "my_source"}],
        "ids": ["id1", "id2", "id3"],
    }
)

query_args = {"text": ["This is a query document", "This is a another query document"], "metric": ["cosine", "l2"]}


# Set up the experiment
experiment = LanceDBExperiment(
    data=data,
    embedding_fns=emb_fns,
    query_args=query_args,
)

# [Optional] Advanced query args
# Our test queries, along with optional query args. LanceDB query accepts a few args to customize your search:
# metrics: "l2", "cosine", or "dot" (cosine by default)
# filter: SQL where clause to filter the vector search results before applying the limit. (None by default)
# limit: number of results to return (3 by default)
"""
query_args_adv = {
                "text": ["This is a query document", "This is a another query document"], 
                "metric": ["cosine", "l2", "dot"],
                "filter": ["text IS NOT NULL" , "text LIKE '%document.%'"]
                }
experiment = LanceDBExperiment(
    data=data,
    embedding_fns=emb_fns,
    query_args=query_args_adv,
 
)
"""

We can then run the experiment to get results.

In [5]:
experiment.run()

We can visualize the result. In this case, the result of the second query "This is a another query document" is different.

paraphrase-MiniLM-L3-v2: [id2, id3, id1]

default (all-MiniLM-L6-v2) : [id2, id1, id3]

In [6]:
experiment.visualize()

,text,metric,emb_fn,top doc ids,distances,documents,latency
0,This is a query document,cosine,openai-ada-002,"[id1, id3, id2]","[0.06923848390579224, 0.08619403839111328, 0.10083281993865967]","[This is a document, This is the document., This is another document]",2.047757
1,This is a query document,l2,openai-ada-002,"[id1, id3, id2]","[0.1383669227361679, 0.17228886485099792, 0.2016243040561676]","[This is a document, This is the document., This is another document]",0.392724
2,This is a another query document,cosine,openai-ada-002,"[id2, id1, id3]","[0.05721437931060791, 0.08627921342849731, 0.1029735803604126]","[This is another document, This is a document, This is the document.]",5.544514
3,This is a another query document,l2,openai-ada-002,"[id2, id1, id3]","[0.11442875862121582, 0.17255812883377075, 0.20594733953475952]","[This is another document, This is a document, This is the document.]",0.622729
4,This is a query document,cosine,minilm_l6,"[id1, id3, id2]","[0.8099705576896667, 0.8289484977722168, 0.8308900594711304]","[This is a document, This is the document., This is another document]",0.198556
5,This is a query document,l2,minilm_l6,"[id1, id3, id2]","[1.619940996170044, 1.6578971147537231, 1.6617801189422607]","[This is a document, This is the document., This is another document]",0.190643
6,This is a another query document,cosine,minilm_l6,"[id1, id3, id2]","[0.8099705576896667, 0.8289484977722168, 0.8308900594711304]","[This is a document, This is the document., This is another document]",0.267542
7,This is a another query document,l2,minilm_l6,"[id1, id3, id2]","[1.619940996170044, 1.6578971147537231, 1.6617801189422607]","[This is a document, This is the document., This is another document]",0.262355
8,This is a query document,cosine,default,"[id2, id3, id1]","[0.7633732557296753, 0.773878812789917, 0.7882261872291565]","[This is another document, This is the document., This is a document]",0.111481
9,This is a query document,l2,default,"[id3, id1, id2]","[45.84406280517578, 49.12738037109375, 49.839256286621094]","[This is the document., This is a document, This is another document]",0.113457


## Evaluate the model response

To evaluate the results, we'll define an evaluation function. Sometimes, you know order of the most relevant document should be given a query, and you can compute the correlation between expected ranking and actual ranking.

In [7]:
import scipy.stats as stats

# For each query, you can define what the expected ranking is.
EXPECTED_RANKING = {
    "This is a query document": ["id1", "id3", "id2"],
    "This is a another query document": ["id2", "id3", "id1"],
}


def measure_correlation(row: "pandas.core.series.Series", ranking_column_name: str = "top doc ids") -> float:
    r"""
    A simple test that compares the expected ranking for a given query with the actual ranking produced
    by the embedding function being tested.
    """
    input_query = row["text"]
    correlation, _ = stats.spearmanr(row[ranking_column_name], EXPECTED_RANKING[input_query])
    return correlation

Finally, we can evaluate and visualize the results.

In [8]:
experiment.evaluate("ranking_correlation", measure_correlation)

In [9]:
experiment.visualize()

,text,metric,emb_fn,top doc ids,distances,documents,latency,ranking_correlation
0,This is a query document,cosine,openai-ada-002,"[id1, id3, id2]","[0.06923848390579224, 0.08619403839111328, 0.10083281993865967]","[This is a document, This is the document., This is another document]",2.047757,1.0
1,This is a query document,l2,openai-ada-002,"[id1, id3, id2]","[0.1383669227361679, 0.17228886485099792, 0.2016243040561676]","[This is a document, This is the document., This is another document]",0.392724,1.0
2,This is a another query document,cosine,openai-ada-002,"[id2, id1, id3]","[0.05721437931060791, 0.08627921342849731, 0.1029735803604126]","[This is another document, This is a document, This is the document.]",5.544514,-1.0
3,This is a another query document,l2,openai-ada-002,"[id2, id1, id3]","[0.11442875862121582, 0.17255812883377075, 0.20594733953475952]","[This is another document, This is a document, This is the document.]",0.622729,-1.0
4,This is a query document,cosine,minilm_l6,"[id1, id3, id2]","[0.8099705576896667, 0.8289484977722168, 0.8308900594711304]","[This is a document, This is the document., This is another document]",0.198556,1.0
5,This is a query document,l2,minilm_l6,"[id1, id3, id2]","[1.619940996170044, 1.6578971147537231, 1.6617801189422607]","[This is a document, This is the document., This is another document]",0.190643,1.0
6,This is a another query document,cosine,minilm_l6,"[id1, id3, id2]","[0.8099705576896667, 0.8289484977722168, 0.8308900594711304]","[This is a document, This is the document., This is another document]",0.267542,0.5
7,This is a another query document,l2,minilm_l6,"[id1, id3, id2]","[1.619940996170044, 1.6578971147537231, 1.6617801189422607]","[This is a document, This is the document., This is another document]",0.262355,0.5
8,This is a query document,cosine,default,"[id2, id3, id1]","[0.7633732557296753, 0.773878812789917, 0.7882261872291565]","[This is another document, This is the document., This is a document]",0.111481,0.5
9,This is a query document,l2,default,"[id3, id1, id2]","[45.84406280517578, 49.12738037109375, 49.839256286621094]","[This is the document., This is a document, This is another document]",0.113457,-1.0


You can also use auto evaluation. We will add an example of this in the near future.